In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
from pyspark.sql.functions import expr, when, trim, col, to_date

In [0]:
silver_ventasbase = (
    spark.table(f"{catalog_name}.{schema_bronze}.ventasbase_raw")  # Lee la tabla raw
    .withColumn("id_venta", expr("try_cast(id_venta as int)"))  # Convierte id_venta a entero
    .withColumn("id_cliente", expr("try_cast(id_cliente as int)"))  # Convierte id_cliente a entero
    .withColumn("id_tienda", expr("try_cast(id_tienda as int)"))  # Convierte id_tienda a entero
    .withColumn("id_producto", expr("try_cast(id_producto as int)"))  # Convierte id_producto a entero
    # Corrige la columna cantidad: si es "diez" la pone en 10, si no intenta convertir a entero
    .withColumn("cantidad", when(trim(col("cantidad")) == "diez", 10)
        .otherwise(expr("try_cast(cantidad as int)"))
    )
    .withColumn("monto", col("monto").cast("double"))  # Convierte monto a double
    # Si el monto es negativo, lo corrige a 0 en monto_corregido
    .withColumn("monto_corregido", when(col("monto") < 0, 0).otherwise(col("monto")))
    # Marca si el monto fue corregido o no
    .withColumn("estado_monto", when(col("monto") < 0, "Corregido").otherwise("No corregido"))
    # Limpia y normaliza la fecha de venta en varios formatos posibles
    .withColumn(
        "fecha_venta",
        when(col("fecha_venta") == "N/A", None)
        .when(col("fecha_venta").rlike("^[0-9]{4}-[0-9]{2}-[0-9]{2}$"), to_date(col("fecha_venta"), "yyyy-MM-dd").cast("timestamp"))
        .when(col("fecha_venta").rlike("^[0-9]{2}-[0-9]{2}-[0-9]{4}$"), to_date(col("fecha_venta"), "MM-dd-yyyy").cast("timestamp"))
        .when(col("fecha_venta").rlike("^[0-9]{2}/[0-9]{2}/[0-9]{4}$"), to_date(col("fecha_venta"), "dd/MM/yyyy").cast("timestamp"))
        .when(col("fecha_venta").rlike("^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}$"), to_date(col("fecha_venta"), "yyyy/MM/dd HH:mm:ss").cast("timestamp"))  
        .otherwise(None)
    )
    # Limpia y normaliza la fecha de actualización en varios formatos posibles
    .withColumn(
        "updated_at",
        when(col("updated_at") == "N/A", None)
        .when(col("updated_at").rlike("^[0-9]{4}-[0-9]{2}-[0-9]{2}$"), to_date(col("updated_at"), "yyyy-MM-dd").cast("timestamp"))
        .when(col("updated_at").rlike("^[0-9]{2}-[0-9]{2}-[0-9]{4}$"), to_date(col("updated_at"), "MM-dd-yyyy").cast("timestamp"))
        .when(col("updated_at").rlike("^[0-9]{2}/[0-9]{2}/[0-9]{4}$"), to_date(col("updated_at"), "dd/MM/yyyy").cast("timestamp"))
        .when(col("updated_at").rlike("^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}$"), to_date(col("updated_at"), "yyyy/MM/dd HH:mm:ss").cast("timestamp"))  
        .otherwise(None)
    )
    .dropDuplicates(["id_venta"])  # Elimina duplicados por id_venta
    .select(
        "id_venta", "id_cliente", "id_tienda", "id_producto", "cantidad",
        "monto", "monto_corregido", "estado_monto",
        "fecha_venta", "updated_at"
    )  # Selecciona solo las columnas relevantes
)

In [0]:
display(silver_ventasbase)

In [0]:
silver_ventasbase.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.ventasbase")